# Gallery for the Overview Paper

The purpose of this notebook is to build a nice gallery of object images for the overview paper.

For future reference: The notebook must be run from https://jupyter-dev.nersc.gov with the following (approximate) activation script:

```bash
#!/bin/bash                                                                                                           
version=$1                                                                                                            
connection_file=$2                                                                                                    

desiconda_version=20170818-1.1.12-img                                                                                 
module use /global/common/${NERSC_HOST}/contrib/desi/desiconda/$desiconda_version/modulefiles                         
module load desiconda                                                                                                 

export LEGACYPIPE_DIR=$SCRATCH/repos/legacypipe                                                                       

export PATH=$LEGACYPIPE_DIR/bin:${PATH}                                                                               
export PATH=$SCRATCH//repos/build/bin:$PATH                                                                           
export PYTHONPATH=$LEGACYPIPE_DIR/py:${PYTHONPATH}                                                                    
export PYTHONPATH=$SCRATCH/repos/build/lib/python3.5/site-packages:$PYTHONPATH                                        

module use $LEGACYPIPE_DIR/bin/modulefiles/cori                                                                       
module load dust                                                                                                      

exec python -m ipykernel -f $connection_file
```

Some neat objects:
* [Bow shock](http://legacysurvey.org/viewer?ra=325.6872&dec=1.0032&zoom=14&layer=decals-dr5)
* [Abell 383](http://legacysurvey.org/viewer?ra=42.0141&dec=-3.5291&zoom=15&layer=decals-dr5)
* [SDSS/C4-2010 Galaxy Cluster](http://legacysurvey.org/viewer?ra=29.0707&dec=1.0510&zoom=13&layer=decals-dr5)
* [NGC2874 Group](http://legacysurvey.org/viewer?ra=141.4373&dec=11.4284&zoom=13&layer=decals-dr5)
* [UGC10321 Group](http://legacysurvey.org/viewer?ra=244.5280&dec=21.5591&zoom=14&layer=decals-dr5)
* [NGC6742 (PN)](http://legacysurvey.org/viewer/?layer=decals-dr6&ra=284.83291667&dec=48.46527778)

### Imports and paths

In [1]:
import os, sys
import shutil, time, warnings
from contextlib import redirect_stdout
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from astropy.table import Table, vstack
from PIL import Image, ImageDraw, ImageFont

In [3]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [4]:
%matplotlib inline

### Preliminaries

Define the data release and the various output directories.

In [5]:
PIXSCALE = 0.262

In [6]:
figdir = os.path.join( os.getenv('SCRATCH'), 'overview-paper')
figfile = os.path.join(figdir, 'gallery.fits')

In [7]:
jpgdir = os.path.join(figdir, 'jpg')
if not os.path.isdir(jpgdir):
    os.mkdir(jpgdir)

In [8]:
pngdir = os.path.join(figdir, 'png')
if not os.path.isdir(pngdir):
    os.mkdir(pngdir)

#### Build a sample with the objects of interest.

In [9]:
cat = Table()
cat['name'] = (
    'NGC6742',
    'M92', 
    'Bow-Shock', 
    'NGC2782',
    'UGC10321', 
    'C4-2010'
)
cat['nicename'] = (
    'NGC 6742 Planetary Nebula',
    'Messier 92 Globular Cluster', 
    'Interstellar Bow Shock', 
    'NGC 2782',
    'UGC 10321 Galaxy Group', 
    'SDSS/C4 Galaxy Cluster 2010'
)
cat['viewer'] = (
    'http://legacysurvey.org/viewer/?layer=decals-dr6&ra=284.83291667&dec=48.46527778',
    'http://legacysurvey.org/viewer/?layer=decals-dr6&ra=259.28029167&dec=43.13652778&zoom=12',
    'http://legacysurvey.org/viewer?ra=325.6872&dec=1.0032&zoom=14&layer=decals-dr5',         
    'http://legacysurvey.org/viewer/?layer=decals-dr6&ra=138.52129167&dec=40.11369444&zoom=12',
    'http://legacysurvey.org/viewer?ra=244.5280&dec=21.5591&zoom=14&layer=decals-dr5',
    'http://legacysurvey.org/viewer?ra=29.0707&dec=1.0510&zoom=13&layer=decals-dr5'
)
cat['dr'] = (
    'dr6',
    'dr6', 
    'dr5',
    'dr6',
    'dr5', 
    'dr5'
)
cat['ra'] = (
    284.83291667,
    259.28029167, 
    325.6872, 
    138.52129167,
    244.5280, 
    29.070641492
)
cat['dec'] = (
    48.46527778,
    43.13652778, 
    1.0032, 
    40.11369444,
    21.5591, 
    1.050816667
)
cat['diam'] = np.array([
    1.5,
    20, 
    4, 
    7,
    3, 
    5 
]).astype('f4') # [arcmin]
cat

name,nicename,viewer,dr,ra,dec,diam
str9,str27,str88,str3,float64,float64,float32
NGC6742,NGC 6742 Planetary Nebula,http://legacysurvey.org/viewer/?layer=decals-dr6&ra=284.83291667&dec=48.46527778,dr6,284.83291667,48.46527778,1.5
M92,Messier 92 Globular Cluster,http://legacysurvey.org/viewer/?layer=decals-dr6&ra=259.28029167&dec=43.13652778&zoom=12,dr6,259.28029167,43.13652778,20.0
Bow-Shock,Interstellar Bow Shock,http://legacysurvey.org/viewer?ra=325.6872&dec=1.0032&zoom=14&layer=decals-dr5,dr5,325.6872,1.0032,4.0
NGC2782,NGC 2782,http://legacysurvey.org/viewer/?layer=decals-dr6&ra=138.52129167&dec=40.11369444&zoom=12,dr6,138.52129167,40.11369444,7.0
UGC10321,UGC 10321 Galaxy Group,http://legacysurvey.org/viewer?ra=244.5280&dec=21.5591&zoom=14&layer=decals-dr5,dr5,244.528,21.5591,3.0
C4-2010,SDSS/C4 Galaxy Cluster 2010,http://legacysurvey.org/viewer?ra=29.0707&dec=1.0510&zoom=13&layer=decals-dr5,dr5,29.070641492,1.050816667,5.0


#### Some rejected objects.

In [10]:
toss = Table()
toss['name'] = (
    'Abell383', 
    'NGC2874'
)
toss['nicename'] = (
    'Abell 383', 
    'NGC2874 Galaxy Group'
)
toss['viewer'] = (
    'http://legacysurvey.org/viewer?ra=42.0141&dec=-3.5291&zoom=15&layer=decals-dr5',
    'http://legacysurvey.org/viewer?ra=141.4373&dec=11.4284&zoom=13&layer=decals-dr5'
)
toss['dr'] = (
    'dr5', # Abell 383
    'dr5'  # C4 cluster
)
toss['ra'] = (
    42.0141, 
    141.44215000
)
toss['dec'] = (
    -3.5291, 
    11.43696000
)
toss['diam'] = np.array([
    6, 
    6
]).astype('f4') # [arcmin]
toss

name,nicename,viewer,dr,ra,dec,diam
str8,str20,str79,str3,float64,float64,float32
Abell383,Abell 383,http://legacysurvey.org/viewer?ra=42.0141&dec=-3.5291&zoom=15&layer=decals-dr5,dr5,42.0141,-3.5291,6.0
NGC2874,NGC2874 Galaxy Group,http://legacysurvey.org/viewer?ra=141.4373&dec=11.4284&zoom=13&layer=decals-dr5,dr5,141.44215,11.43696,6.0


### Ensure all objects are in the DR5+DR6 footprint before building coadds.

In [11]:
def init_survey(dr='dr5'):
    from legacypipe.survey import LegacySurveyData
    
    survey = ''
    try:
        del survey
    except:
        pass
    
    if dr == 'dr5':
        survey = LegacySurveyData(
            #survey_dir='/global/cscratch1/sd/dstn/dr5-new-sky',
            survey_dir='/global/cscratch1/sd/desiproc/dr7',
            #survey_dir='/global/project/projectdirs/cosmo/work/legacysurvey/dr5',
            #cache_dir='/global/cscratch1/sd/desiproc/dr5-calib',
            #survey_dir='/global/cscratch1/sd/desiproc/dr5-calib',
            #cache_dir='/global/cscratch1/sd/desiproc/dr7/calib',
            output_dir=figdir)
    elif dr == 'dr6':
        survey = LegacySurveyData(
            survey_dir='/global/cscratch1/sd/dstn/dr6plus',
            output_dir=figdir)
    
    return survey

In [12]:
def simple_wcs(obj):
    """Build a simple WCS object for a single object."""
    from astrometry.util.util import Tan
    
    size = np.rint(obj['diam'] * 60 / PIXSCALE).astype('int') # [pixels]
    wcs = Tan(obj['ra'], obj['dec'], size/2+0.5, size/2+0.5,
                 -PIXSCALE/3600.0, 0.0, 0.0, PIXSCALE/3600.0, 
                 float(size), float(size))
    return wcs

In [13]:
def _build_sample_one(args):
    """Wrapper function for the multiprocessing."""
    return build_sample_one(*args)

In [14]:
def build_sample_one(obj, verbose=False):
    """Wrapper function to find overlapping grz CCDs for a given object.
    
    """
    survey = init_survey(dr=obj['dr'])
    
    print('Working on {}...'.format(obj['name']))
    wcs = simple_wcs(obj)
    try:
        ccds = survey.ccds_touching_wcs(wcs) # , ccdrad=2*diam/3600)
    except:
        return None
    
    if ccds:
        # Is there 3-band coverage?
        if 'g' in ccds.filter and 'r' in ccds.filter and 'z' in ccds.filter:
            if verbose:
                print('For {} found {} CCDs, RA = {:.5f}, Dec = {:.5f}, Diameter={:.4f} arcmin'.format(
                        obj['name'], len(ccds), obj['ra'], obj['dec'], obj['diam']))
            return obj
    return None

In [15]:
def build_sample(cat, use_nproc=nproc):
    """Build the full sample with grz coverage in DR6."""

    sampleargs = list()
    for cc in cat:
        sampleargs.append( (cc, True) ) # the False refers to verbose=False

    if use_nproc > 1:
        p = multiprocessing.Pool(nproc)
        result = p.map(_build_sample_one, sampleargs)
        p.close()
    else:
        result = list()
        for args in sampleargs:
            result.append(_build_sample_one(args))

    # Remove non-matching objects and write out the sample
    outcat = vstack(list(filter(None, result)))
    print('Found {}/{} objects in the DR5+6 footprint.'.format(len(outcat), len(cat)))
    
    return outcat

In [16]:
sample = build_sample(cat, use_nproc=1)

Working on NGC6742...
Searching /global/cscratch1/sd/dstn/dr6plus/survey-ccds-dr6plus.kd.fits
162 CCDs within 1.0 deg of RA,Dec (284.833, 48.465)
For NGC6742 found 7 CCDs, RA = 284.83292, Dec = 48.46528, Diameter=1.5000 arcmin
Working on M92...
Searching /global/cscratch1/sd/dstn/dr6plus/survey-ccds-dr6plus.kd.fits
192 CCDs within 1.0 deg of RA,Dec (259.280, 43.137)
For M92 found 33 CCDs, RA = 259.28029, Dec = 43.13653, Diameter=20.0000 arcmin
Working on Bow-Shock...
Searching /global/cscratch1/sd/desiproc/dr7/survey-ccds-dr7.kd.fits
4923 CCDs within 1.0 deg of RA,Dec (325.687, 1.003)
For Bow-Shock found 176 CCDs, RA = 325.68720, Dec = 1.00320, Diameter=4.0000 arcmin
Working on NGC2782...
Searching /global/cscratch1/sd/dstn/dr6plus/survey-ccds-dr6plus.kd.fits
195 CCDs within 1.0 deg of RA,Dec (138.521, 40.114)
For NGC2782 found 13 CCDs, RA = 138.52129, Dec = 40.11369, Diameter=7.0000 arcmin
Working on UGC10321...
Searching /global/cscratch1/sd/desiproc/dr7/survey-ccds-dr7.kd.fits
613 C

In [17]:
print('Writing {}'.format(figfile))
sample.write(figfile, overwrite=True)

Writing /global/cscratch1/sd/ioannis/overview-paper/gallery.fits


In [18]:
sample

name,nicename,viewer,dr,ra,dec,diam
str36,str108,str352,str12,float64,float64,float32
NGC6742,NGC 6742 Planetary Nebula,http://legacysurvey.org/viewer/?layer=decals-dr6&ra=284.83291667&dec=48.46527778,dr6,284.83291667,48.46527778,1.5
M92,Messier 92 Globular Cluster,http://legacysurvey.org/viewer/?layer=decals-dr6&ra=259.28029167&dec=43.13652778&zoom=12,dr6,259.28029167,43.13652778,20.0
Bow-Shock,Interstellar Bow Shock,http://legacysurvey.org/viewer?ra=325.6872&dec=1.0032&zoom=14&layer=decals-dr5,dr5,325.6872,1.0032,4.0
NGC2782,NGC 2782,http://legacysurvey.org/viewer/?layer=decals-dr6&ra=138.52129167&dec=40.11369444&zoom=12,dr6,138.52129167,40.11369444,7.0
UGC10321,UGC 10321 Galaxy Group,http://legacysurvey.org/viewer?ra=244.5280&dec=21.5591&zoom=14&layer=decals-dr5,dr5,244.528,21.5591,3.0
C4-2010,SDSS/C4 Galaxy Cluster 2010,http://legacysurvey.org/viewer?ra=29.0707&dec=1.0510&zoom=13&layer=decals-dr5,dr5,29.070641492,1.050816667,5.0


### Generate the color mosaics for each object.

In [19]:
def custom_brickname(obj, prefix='custom-'): 
    brickname = 'custom-{:06d}{}{:05d}'.format(
        int(1000*obj['ra']), 'm' if obj['dec'] < 0 else 'p', 
        int(1000*np.abs(obj['dec'])))
    return brickname

In [23]:
def custom_coadds_one(obj, scale=PIXSCALE, clobber=False):
    from legacypipe.runbrick import run_brick
    #from astrometry.util.multiproc import multiproc
    #from legacypipe.runbrick import stage_tims, run_brick
    #from legacypipe.coadds import make_coadds

    name = obj['name']
    jpgfile = os.path.join(jpgdir, '{}.jpg'.format(name))
    if os.path.isfile(jpgfile) and not clobber:
        print('File {} exists...skipping.'.format(jpgfile))
    else:
        size = np.rint(obj['diam'] * 60 / scale).astype('int') # [pixels]
        print('Generating mosaic for {} with width={} pixels.'.format(name, size))
        
        bands = ('g', 'r', 'z')
        survey = init_survey(dr=obj['dr'])
        brickname = custom_brickname(obj, prefix='custom-')
        
        #mp = multiproc(nthreads=nproc)
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            run_brick(None, survey, radec=(obj['ra'], obj['dec']), pixscale=scale, 
                      width=size, height=size, stages=['image_coadds'], splinesky=True,
                      early_coadds=True, pixPsf=True, hybridPsf=True, normalizePsf=True,
                      write_pickles=False, depth_cut=False, apodize=True, threads=nproc,
                      do_calibs=False, ceres=False)

            #P = stage_tims(ra=obj['ra'], dec=obj['dec'], brickname=brickname,
            #               survey=survey, W=size, H=size, pixscale=scale,
            #               mp=mp, pixPsf=True, hybridPsf=True, normalizePsf=True,
            #               depth_cut=False, apodize=True, do_calibs=False, 
            #               rex=True, splinesky=True)

            #C = make_coadds(P['tims'], bands, P['targetwcs'], mp=mp,
            #                callback=write_coadd_images,
            #                callback_args=(survey, brickname, P['version_header'],
            #                               P['tims'], P['targetwcs']))
            
        sys.stdout.flush()    
        _jpgfile = os.path.join(survey.output_dir, 'coadd', 'cus', brickname, 
                               'legacysurvey-{}-image.jpg'.format(brickname))
        shutil.copy(_jpgfile, jpgfile)
        #shutil.rmtree(os.path.join(survey.output_dir, 'coadd'))

In [21]:
custom_coadds_one(sample[2], clobber=True)

Generating mosaic for Bow-Shock with width=916 pixels.
Total Memory Available to Job:
Maximum VMEM                      (RLIMIT_AS      ) :                   -1                   -1
Maximum core file size            (RLIMIT_CORE    ) :                    0                   -1
Maximum CPU time                  (RLIMIT_CPU     ) :                   -1                   -1
Maximum file size                 (RLIMIT_FSIZE   ) :                   -1                   -1
Maximum heap size                 (RLIMIT_DATA    ) :                   -1                   -1
Maximum stack size                (RLIMIT_STACK   ) :                   -1                   -1
Maximum resident set size         (RLIMIT_RSS     ) :                   -1                   -1
Maximum number of processes       (RLIMIT_NPROC   ) :                 2048                 8192
Maximum number of open files      (RLIMIT_NOFILE  ) :                 4096                65536
Maximum lockable memory address   (RLIMIT_MEMLOCK 

Applying surface-brightness scaling of 0.998 to decam-00241586-S31 r
Applying surface-brightness scaling of 0.997 to decam-00450862-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450858-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450863-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450868-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450878-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450839-N9 r
Applying surface-brightness scaling of 0.998 to decam-00241589-S28 r
Applying surface-brightness scaling of 0.998 to decam-00241588-S28 r
Applying surface-brightness scaling of 0.997 to decam-00450854-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450849-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450869-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450855-N9 r
Applying surface-brightness scaling of 0.997 to decam-00450865-N9 r
Applying surface-brightness scaling of 0.997 

Wrote /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/tmp-legacysurvey-custom-325687p01003-image-z.fits.fz
Renamed to /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/legacysurvey-custom-325687p01003-image-z.fits.fz
Wrote /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/tmp-legacysurvey-custom-325687p01003-invvar-z.fits.fz
Renamed to /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/legacysurvey-custom-325687p01003-invvar-z.fits.fz
Wrote /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/tmp-legacysurvey-custom-325687p01003-nexp-z.fits.fz
Renamed to /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/legacysurvey-custom-325687p01003-nexp-z.fits.fz
Wrote /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/custom-325687p01003/tmp-legacysurvey-custom-325687p01003-depth-z.fits.fz
Renamed to /global/cscratch1/sd/ioannis/overview-paper/coadd/cus/c

In [22]:
stop

NameError: name 'stop' is not defined

In [ ]:
def custom_coadds(sample, clobber=False):
    for obj in sample:
        custom_coadds_one(obj, clobber=clobber)

In [ ]:
coaddslogfile = os.path.join(figdir, 'make-coadds.log')
print('Generating the coadds.')
print('Logging to {}'.format(coaddslogfile))
t0 = time.time()
with open(coaddslogfile, 'w') as log:
    with redirect_stdout(log):
        make_coadds(sample, clobber=True)
print('Total time = {:.3f} minutes.'.format((time.time() - t0) / 60))

### Add labels and a scale bar.

In [ ]:
barlen = np.round(60.0 / PIXSCALE).astype('int')
fonttype = os.path.join(figdir, 'Georgia.ttf')

In [ ]:
def _add_labels_one(args):
    """Wrapper function for the multiprocessing."""
    return add_labels_one(*args)

In [ ]:
def add_labels_one(obj, verbose=False):
    name = obj['name']
    nicename = obj['nicename']
    
    jpgfile = os.path.join(jpgdir, '{}.jpg'.format(name))
    pngfile = os.path.join(pngdir, '{}.png'.format(name))
    thumbfile = os.path.join(pngdir, 'thumb-{}.png'.format(name))
        
    im = Image.open(jpgfile)
    sz = im.size
    fntsize = np.round(sz[0]/28).astype('int')
    width = np.round(sz[0]/175).astype('int')
    font = ImageFont.truetype(fonttype, size=fntsize)
    draw = ImageDraw.Draw(im)
    # Label the object name--
    draw.text((0+fntsize*2, 0+fntsize*2), nicename, font=font)
    # Add a scale bar--
    x0, x1, yy = sz[1]-fntsize*2-barlen, sz[1]-fntsize*2, sz[0]-fntsize*2
    draw.line((x0, yy, x1, yy), fill='white', width=width)
    im.save(pngfile)    
        
    # Generate a thumbnail
    if False:
        cmd = '/usr/bin/convert -thumbnail 300x300 {} {}'.format(pngfile, thumbfile)
        os.system(cmd)

In [ ]:
def add_labels(sample):
    labelargs = list()
    for obj in sample:
        labelargs.append((obj, False))

    if nproc > 1:
        p = multiprocessing.Pool(nproc)
        res = p.map(_add_labels_one, labelargs)
        p.close()
    else:
        for args in labelargs:
            res = _add_labels_one(args)

In [ ]:
%time add_labels(sample)

### Finally make a nice montage figure for the paper.

In [ ]:
def make_montage(cat, clobber=False):
    montagefile = os.path.join(figdir, 'overview-gallery.png')

    ncol = 3
    nrow = np.ceil(len(sample) / ncol).astype('int')
    
    if not os.path.isfile(montagefile) or clobber:
        cmd = 'montage -bordercolor white -borderwidth 1 -tile {}x{} -geometry 512x512 '.format(ncol, nrow)
        cmd = cmd+' '.join([os.path.join(pngdir, '{}.png'.format(name)) for name in cat['name']])
        cmd = cmd+' {}'.format(montagefile)
        print(cmd)
        os.system(cmd)        
        print('Writing {}'.format(montagefile))

In [ ]:
%time make_montage(cat, clobber=True)